In [7]:
# ! pip install openpyxl
# ! hadoop fs -mkdir /resources
# ! wget  -nd  biobank.ndph.ox.ac.uk/ukb/ukb/auxdata/omop_concept.tsv  && hadoop fs -put omop_concept.tsv /resources/
# ! wget  -nd  biobank.ndph.ox.ac.uk/ukb/ukb/auxdata/omop_drug_strength.tsv && hadoop fs -put omop_drug_strength.tsv /resources/
# ! wget https://zenodo.org/records/6949696/files/OMOP2OBO_V1_Drug_Exposure_Mapping_Oct2020.xlsx
! hadoop fs -put RXNCONSO.RRF /resources/

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/cluster/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/cluster/dnax/jars/dnanexus-api-0.1.0-SNAPSHOT-jar-with-dependencies.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
2024-08-29 12:44:58,308 WARN metrics.MetricsReporter: Unable to initialize metrics scraping configurations from hive-site.xml. Message:InputStream cannot be null
2024-08-29 12:44:58,423 WARN service.DNAxApiSvc: Using default configurations. Unable to find dnanexus.conf.location=null
2024-08-29 12:44:58,423 INFO service.DNAxApiSvc: apiserver connection-pool config. MaxPoolSize=10, MaxPoolPerRoute=10,MaxWaitTimeout=60000
2024-08-29 12:44:58,423 INFO service.DNAxApiSvc: initializing http connection man

In [2]:
# NOTES
# https://www.nlm.nih.gov/research/umls/rxnorm/docs/appendix1.html
# https://www.nlm.nih.gov/research/umls/rxnorm/docs/appendix5.html
# https://www.nlm.nih.gov/research/umls/rxnorm/docs/techdoc.html#conso

In [1]:
import pyspark
import dxpy
import dxdata
import pandas as pd
import subprocess
from pathlib import Path
from pyspark.sql.functions import col

In [2]:
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

dispensed_database_name = dxpy.find_one_data_object(classname="database", name="app*", folder="/", name_mode="glob", describe=True)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(typename="Dataset", name="app*.dataset", folder="/", name_mode="glob")["id"]

spark.sql("USE " + dispensed_database_name)

dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [3]:
# Init pyspark and load dataframe
OMOP_TABLE = "omop_drug_era" # 'omop_drug_era', 'omop_drug_exposure'

# Load dose era table
ode = dataset[OMOP_TABLE]

field_names = [f.name for f in ode.fields]

df = ode.retrieve_fields(names=field_names, engine=dxdata.connect())
df = df.drop("eid")

print(f"Number of entries {df.count()}")
df.show(5)

Number of entries 19959413
+-------------+---------------+-------------------+-----------------+-------------------+--------+
|  drug_era_id|drug_concept_id|drug_era_start_date|drug_era_end_date|drug_exposure_count|gap_days|
+-------------+---------------+-------------------+-----------------+-------------------+--------+
|1236950609195|       19005129|         12/05/2014|       10/06/2014|                  1|       0|
| 798863919105|         743670|         30/03/2011|       02/09/2011|                  5|      12|
| 721554547993|       19008994|         12/10/2010|       07/11/2010|                  1|       0|
| 910533073010|         755695|         23/10/2006|       04/10/2007|                  6|      23|
| 317827646206|       19010400|         30/07/1999|       28/08/1999|                  1|       0|
+-------------+---------------+-------------------+-----------------+-------------------+--------+
only showing top 5 rows



In [4]:
# Get RxNorm Ingredients (IN) and associated concept_codes
conc = spark.read.csv("/resources/omop_concept.tsv", sep = "\t", header=True, inferSchema=True)
rx_ing = conc.filter(
    (conc["vocabulary_id"] == "RxNorm") & (conc["concept_class_id"] == "Ingredient") & (conc["invalid_reason"].isNull())
)
print(f"RxNorm Ingredients: {rx_ing.count()}")
print(rx_ing.columns)
rx_ing.show(3, truncate = False, vertical = False)

RxNorm Ingredients: 13137
['concept_id', 'concept_name', 'domain_id', 'vocabulary_id', 'concept_class_id', 'standard_concept', 'concept_code', 'valid_start_date', 'valid_end_date', 'invalid_reason']
+----------+------------------------------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+
|concept_id|concept_name                        |domain_id|vocabulary_id|concept_class_id|standard_concept|concept_code|valid_start_date|valid_end_date|invalid_reason|
+----------+------------------------------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+
|501343    |hepatitis B immune globulin         |Drug     |RxNorm       |Ingredient      |S               |26744       |1970-01-01      |2099-12-31    |null          |
|507832    |herpesvirus 3, human                |Drug     |RxNorm       |Ingredient      |S               |11131       |1970-01-0

In [5]:
# Get distinct drug_concept_ids

dci = df.select(["drug_concept_id"]).distinct()
print(f"Number of records before join: {dci.count()}")

dci = dci.join(rx_ing, dci["drug_concept_id"] == rx_ing["concept_id"], how = 'inner')
print(f"Number of records after join: {dci.count()}")

dci.show(5, truncate = False, vertical = False)

Number of records before join: 1620
Number of records after join: 1572
+---------------+----------+------------------------------------------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+
|drug_concept_id|concept_id|concept_name                                    |domain_id|vocabulary_id|concept_class_id|standard_concept|concept_code|valid_start_date|valid_end_date|invalid_reason|
+---------------+----------+------------------------------------------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+
|501343         |501343    |hepatitis B immune globulin                     |Drug     |RxNorm       |Ingredient      |S               |26744       |1970-01-01      |2099-12-31    |null          |
|509081         |509081    |meningococcal group C polysaccharide            |Drug     |RxNorm       |Ingredient      |S               |29503     

In [8]:
###
### RxNorm Ingredient
###

# Documentation here https://www.nlm.nih.gov/research/umls/rxnorm/docs/techdoc.html#conso
# See https://www.nlm.nih.gov/research/umls/rxnorm/docs/appendix5.html for TTY
# Essentially _c11 = SAB, Source abbreviation. _c12 = TTY, Term type in source, _c16 = SUPPRESS, where N is No

conso = spark.read.csv("/resources/RXNCONSO.RRF", sep = "|", header=False, inferSchema=True)

concept_code_to_drug = conso.filter(
    (conso["_c11"] == "RXNORM") & (conso["_c12"] == "IN") & (conso["_c16"] == "N")
)


# Identify columns where all values are null
non_null_columns = [col_name for col_name in concept_code_to_drug.columns if concept_code_to_drug.filter(col(col_name).isNotNull()).count() > 0]

# Select only the columns that are not entirely null
concept_code_to_drug = concept_code_to_drug.select(*non_null_columns)

print(f"Number of RxNorm IN concepts: {concept_code_to_drug.count()}")
concept_code_to_drug.show(3, truncate = False, vertical = False)

Number of RxNorm IN concepts: 14395
+---+---+--------+--------+---+------+----+----+----------------+----+----+
|_c0|_c1|_c7     |_c8     |_c9|_c11  |_c12|_c13|_c14            |_c16|_c17|
+---+---+--------+--------+---+------+----+----+----------------+----+----+
|44 |ENG|12251526|12251526|44 |RXNORM|IN  |44  |mesna           |N   |4096|
|61 |ENG|12254378|12254378|61 |RXNORM|IN  |61  |beta-alanine    |N   |4096|
|73 |ENG|12252051|12252051|73 |RXNORM|IN  |73  |docosahexaenoate|N   |4096|
+---+---+--------+--------+---+------+----+----+----------------+----+----+
only showing top 3 rows



In [9]:
dci2 = dci.join(concept_code_to_drug, dci["concept_code"] == concept_code_to_drug["_c0"], how = 'inner')
print(dci2.count()) # We lose Senna Leaves which is obsolete
dci2.show(3, truncate = False, vertical = False)

1568
+---------------+----------+-------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+---+---+--------+--------+---+------+----+----+-------------+----+----+
|drug_concept_id|concept_id|concept_name |domain_id|vocabulary_id|concept_class_id|standard_concept|concept_code|valid_start_date|valid_end_date|invalid_reason|_c0|_c1|_c7     |_c8     |_c9|_c11  |_c12|_c13|_c14         |_c16|_c17|
+---------------+----------+-------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+---+---+--------+--------+---+------+----+----+-------------+----+----+
|1319998        |1319998   |acebutolol   |Drug     |RxNorm       |Ingredient      |S               |149         |1970-01-01      |2099-12-31    |null          |149|ENG|12253229|12253229|149|RXNORM|IN  |149 |acebutolol   |N   |4096|
|19024063       |19024063  |acenocoumarol|Drug     |RxNorm       |I

In [10]:
print(f"Number of entries original: {df.count()}")

print(f"Number of entries finished: {df.join(dci2, on = 'drug_concept_id', how = 'inner').count()}")

Number of entries original: 19959413
Number of entries finished: 19833183


In [11]:
dci.join(dci2, on = 'drug_concept_id', how = 'left_anti').show()

+---------------+----------+--------------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+
|drug_concept_id|concept_id|        concept_name|domain_id|vocabulary_id|concept_class_id|standard_concept|concept_code|valid_start_date|valid_end_date|invalid_reason|
+---------------+----------+--------------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+
|       42899447|  42899447|       insulin, pork|     Drug|       RxNorm|      Ingredient|               S|     1309342|      2012-10-01|    2099-12-31|          null|
|         992409|    992409|        Senna leaves|     Drug|       RxNorm|      Ingredient|               S|      237929|      1970-01-01|    2099-12-31|          null|
|       19058973|  19058973|calcium phosphate...|     Drug|       RxNorm|      Ingredient|               S|       47627|      1970-01-01|    2099-12-31|        

In [12]:
dci2.toPandas().to_csv("drug_era_ingredients.tsv", sep = "\t")

In [13]:
###
### ATC codes
###
conso = spark.read.csv("/resources/RXNCONSO.RRF", sep = "|", header=False, inferSchema=True)

concept_code_to_atc = conso.filter(
    (conso["_c11"] == "ATC") & (conso["_c12"] == "IN") & (conso["_c16"] == "N")
)


# Identify columns where all values are null
non_null_columns = [col_name for col_name in concept_code_to_atc.columns if concept_code_to_atc.filter(col(col_name).isNotNull()).count() > 0]

# Select only the columns that are not entirely null
concept_code_to_atc = concept_code_to_atc.select(*non_null_columns)

print(f"Number of ATC codes: {concept_code_to_atc.count()}")
concept_code_to_atc.show(3, truncate = False, vertical = False)

Number of ATC codes: 5495
+---+---+-------+----+----+-------+--------+----+
|_c0|_c1|_c7    |_c11|_c12|_c13   |_c14    |_c16|
+---+---+-------+----+----+-------+--------+----+
|44 |ENG|5481074|ATC |IN  |R05CB05|mesna   |N   |
|44 |ENG|5481075|ATC |IN  |V03AF01|mesna   |N   |
|63 |ENG|5481013|ATC |IN  |C10AX05|meglutol|N   |
+---+---+-------+----+----+-------+--------+----+
only showing top 3 rows



In [14]:
dci3 = dci.join(concept_code_to_atc, dci["concept_code"] == concept_code_to_atc["_c0"], how = 'inner')

print(dci3.count())
dci3.show(3, truncate = False, vertical = False)

1617
+---------------+----------+-------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+---+---+-------+----+----+-------+-------------+----+
|drug_concept_id|concept_id|concept_name |domain_id|vocabulary_id|concept_class_id|standard_concept|concept_code|valid_start_date|valid_end_date|invalid_reason|_c0|_c1|_c7    |_c11|_c12|_c13   |_c14         |_c16|
+---------------+----------+-------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+---+---+-------+----+----+-------+-------------+----+
|1319998        |1319998   |acebutolol   |Drug     |RxNorm       |Ingredient      |S               |149         |1970-01-01      |2099-12-31    |null          |149|ENG|5478462|ATC |IN  |C07AB04|acebutolol   |N   |
|19024063       |19024063  |acenocoumarol|Drug     |RxNorm       |Ingredient      |S               |154         |1970-01-01      |2099-12-3

In [15]:
print(f"Number of entries original: {df.count()}")

print(f"Number of entries finished: {df.join(dci3, on = 'drug_concept_id', how = 'inner').count()}")
print(f"Number of distinct finished: {df.join(dci3, on = 'drug_concept_id', how = 'inner').select('drug_concept_id').distinct().count()}")

Number of entries original: 19959413
Number of entries finished: 38757289
Number of distinct finished: 1141


In [16]:
atc_missing = dci.join(dci3, on = 'drug_concept_id', how = 'left_anti')

print(f"Missing ATC: {atc_missing.count()}")
print(f"Records with ATC codes: {df.join(atc_missing, on = 'drug_concept_id', how = 'left_anti').count()}")


SHOW_MISSING = False
N_SHOW = 10

if SHOW_MISSING:
    dci_anti = atc_missing.show(N_SHOW)

Missing ATC: 431
Records with ATC codes: 17348853


In [18]:
dci3.toPandas().to_csv("drug_era_atc.tsv", sep = "\t")

In [17]:
###
### DrugBank
###
conso = spark.read.csv("/resources/RXNCONSO.RRF", sep = "|", header=False, inferSchema=True)

concept_code_to_db = conso.filter(
    (conso["_c11"] == "DRUGBANK") & (conso["_c12"] == "IN") & (conso["_c16"] == "N")
)


# Identify columns where all values are null
non_null_columns = [col_name for col_name in concept_code_to_db.columns if concept_code_to_db.filter(col(col_name).isNotNull()).count() > 0]

# Select only the columns that are not entirely null
concept_code_to_db = concept_code_to_db.select(*non_null_columns)

print(f"Number of DrugBank identifier: {concept_code_to_db.count()}")
concept_code_to_db.show(3, truncate = False, vertical = False)


Number of DrugBank identifier: 10715
+---+---+--------+-------+--------+----+-------+------------------------------+----+
|_c0|_c1|_c7     |_c9    |_c11    |_c12|_c13   |_c14                          |_c16|
+---+---+--------+-------+--------+----+-------+------------------------------+----+
|48 |ENG|12855631|DB18029|DRUGBANK|IN  |DB18029|24,25-Dihydroxycholecalciferol|N   |
|60 |ENG|10894803|DB01509|DRUGBANK|IN  |DB01509|Tenamfetamine                 |N   |
|74 |ENG|9733930 |DB02362|DRUGBANK|IN  |DB02362|Aminobenzoic acid             |N   |
+---+---+--------+-------+--------+----+-------+------------------------------+----+
only showing top 3 rows



In [18]:
dci4 = dci.join(concept_code_to_db, dci["concept_code"] == concept_code_to_db["_c0"], how = 'inner')

print(dci4.count()) 
dci4.show(3, truncate = False, vertical = False)

1280
+---------------+----------+-------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+---+---+-------+-------+--------+----+-------+-------------+----+
|drug_concept_id|concept_id|concept_name |domain_id|vocabulary_id|concept_class_id|standard_concept|concept_code|valid_start_date|valid_end_date|invalid_reason|_c0|_c1|_c7    |_c9    |_c11    |_c12|_c13   |_c14         |_c16|
+---------------+----------+-------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+---+---+-------+-------+--------+----+-------+-------------+----+
|1319998        |1319998   |acebutolol   |Drug     |RxNorm       |Ingredient      |S               |149         |1970-01-01      |2099-12-31    |null          |149|ENG|8575333|DB01193|DRUGBANK|IN  |DB01193|Acebutolol   |N   |
|19024063       |19024063  |acenocoumarol|Drug     |RxNorm       |Ingredient      |S       

In [19]:
print(f"Number of entries original: {df.count()}")
print(f"Number of entries finished: {df.join(dci4, on = 'drug_concept_id', how = 'inner').count()}")
print(f"Number of distinct finished: {df.join(dci4, on = 'drug_concept_id', how = 'inner').select('drug_concept_id').distinct().count()}")

drugbank_missing = dci.join(dci4, on = 'drug_concept_id', how = 'left_anti')

print(f"Missing DrugBank ID: {drugbank_missing.count()}")
print(f"Records with DrugBank codes: {df.join(drugbank_missing, on = 'drug_concept_id', how = 'left_anti').count()}")


SHOW_MISSING = True
N_SHOW = 10

if SHOW_MISSING:
    dci_anti = drugbank_missing.show(N_SHOW)

Number of entries original: 19959413
Number of entries finished: 18211458
Number of distinct finished: 1271
Missing DrugBank ID: 301
Records with DrugBank codes: 18192505
+---------------+----------+------------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+
|drug_concept_id|concept_id|      concept_name|domain_id|vocabulary_id|concept_class_id|standard_concept|concept_code|valid_start_date|valid_end_date|invalid_reason|
+---------------+----------+------------------+---------+-------------+----------------+----------------+------------+----------------+--------------+--------------+
|         790253|    790253|       clorazepate|     Drug|       RxNorm|      Ingredient|               S|        2353|      1970-01-01|    2099-12-31|          null|
|         900017|    900017|   cascara sagrada|     Drug|       RxNorm|      Ingredient|               S|       66869|      1970-01-01|    2099-12-31|          null|

In [20]:
dci4.toPandas().to_csv("drug_era_drugbank.tsv", sep = "\t")

In [21]:
###
### Chebi / OMOP2OBO
###

chebi = pd.read_excel("OMOP2OBO_V1_Drug_Exposure_Mapping_Oct2020.xlsx", sheet_name="OMOP2OBO_ChEBI_Mapping_Results")
chebi['ONTOLOGY_LOGIC'] = chebi['ONTOLOGY_LOGIC'].astype(str)
chebi = spark.createDataFrame(chebi)

/cluster/spark/python/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [22]:
dci = df.select(["drug_concept_id"]).distinct()
dci5 = dci.join(chebi, dci["drug_concept_id"] == chebi["CONCEPT_ID"], how = "inner")
print(dci5.count())
dci5.show()

1562
+---------------+----------+------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|drug_concept_id|CONCEPT_ID|CONCEPT_CODE|        CONCEPT_NAME|ONTOLOGY_LOGIC|        ONTOLOGY_URI|      ONTOLOGY_LABEL|    MAPPING_CATEGORY|    MAPPING_EVIDENCE|
+---------------+----------+------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|         523283|    523283|    763096.0|poliovirus vaccin...|           nan|         CHEBI_60816|           immunogen|Manual One-to-One...|Source:DrugBank_D...|
|         529303|    529303|    798304.0|diphtheria toxoid...|           nan|         CHEBI_60816|           immunogen|Manual One-to-One...|        Hand Mapping|
|         529411|    529411|    798306.0|tetanus toxoid va...|           nan|         CHEBI_60816|           immunogen|Manual One-to-One...|Source:DrugBank_D...|
|         532881|    53

In [23]:
dci5.toPandas().to_csv("drug_era_chebi.tsv", sep = "\t")